In [1]:
import pandas as pd
import numpy as np
import torch
import torch.utils.data as Data
from torch.autograd import Variable
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler() # define min max scaler

In [2]:
# read file
stock_path = "C:/Users/USER/Desktop/Lab/lab_minmax.csv"
stock_df = pd.read_csv(stock_path)
stock_df=stock_df.fillna(0)

In [3]:
train_set = stock_df[:2200]
test_set = stock_df[2200:]
print(train_set.shape)
print(test_set.shape)

(2200, 12)
(238, 12)


In [4]:
# every features in range(0 , 1)
train_set.describe()

,K value,D value,MACD,William,SMA,WMA,Momentum,RSI,A/D 0,CCI,Close r/f
count,2200.000000,2200.000000,2200.000000,2200.000000,2200.000000,2200.000000,2200.000000,2200.000000,2200.000000,2200.000000,2200.000000
mean,0.478459,0.494744,0.515782,0.515950,0.425581,0.426475,0.464374,0.492458,0.445859,0.463808,1.000000
std,0.242761,0.231248,0.128572,0.292113,0.297391,0.296803,0.101128,0.251304,0.278231,0.159410,0.964113
min,0.004943,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.272646,0.306148,0.453617,0.258398,0.180373,0.181665,0.419330,0.292431,0.208334,0.351748,0.000000
50%,0.469031,0.482729,0.512733,0.520003,0.364709,0.364416,0.461561,0.500000,0.415185,0.467025,1.000000
75%,0.691137,0.682273,0.585561,0.777773,0.712705,0.712135,0.514771,0.687486,0.687456,0.572115,2.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000


In [5]:
# print the number of rise , flat , fall data
print("Train set Rise: %.4f"%(len(train_set[train_set["Close r/f"] == 2])*100 / len(train_set)))
print("Train set Flat: %.4f"%(len(train_set[train_set["Close r/f"] == 1])*100 / len(train_set)))
print("Train set Fall: %.4f"%(len(train_set[train_set["Close r/f"] == 0])*100 / len(train_set)))
print("Test set Rise: %.4f"%(len(test_set[test_set["Close r/f"] == 2])*100 / len(test_set)))
print("Test set Flat: %.4f"%(len(test_set[test_set["Close r/f"] == 1])*100 / len(test_set)))
print("Test set Fall: %.4f"%(len(test_set[test_set["Close r/f"] == 0])*100 / len(test_set)))


Train set Rise: 46.4545
Train set Flat: 7.0909
Train set Fall: 46.4545
Test set Rise: 49.5798
Test set Flat: 2.5210
Test set Fall: 47.8992


In [6]:
# declear training features data
features = []
for i in range(30,len(train_set)):
    x = train_set[i-30:i][["K value ","D value","William","MACD","SMA","WMA","Momentum","RSI","A/D 0","CCI"]].values
    features.append(x.tolist())
features = torch.FloatTensor(features)
features.size()

torch.Size([2170, 30, 10])

In [7]:
# declear trainging labels data
labels = []
for i in range(31,len(train_set)+1):
    x = train_set[i-1:i]["Close r/f"]
    labels.append(x.tolist())
labels = torch.LongTensor(labels).view(-1)
labels.size()

torch.Size([2170])

In [8]:
#hy parameter
torch.manual_seed(1)
EPOCH = 50000
BATCH_SIZE = 16
TIME_STEP = 30
INPUT_SIZE = 10
LR = 0.001

In [9]:

#Mini-Batch

torch_dataset = Data.TensorDataset(features,labels)
train_loader = Data.DataLoader(
    dataset = torch_dataset,
    batch_size = BATCH_SIZE,
    num_workers = 2,
    shuffle=True,
)

In [10]:

#define NN architecture


class RNN(torch.nn.Module):   
    def __init__(self):
        super(RNN,self).__init__()
        # define lstm layer
        self.lstm = torch.nn.LSTM(
            input_size=INPUT_SIZE,
            hidden_size=64,         
            num_layers=1,
            batch_first=True, 
        )
        # dropout layer
        self.dropout = torch.nn.Dropout(p = 0.3)
        # Dense layer
        self.hidden1 = torch.nn.Linear(64, 16)
        self.out = torch.nn.Linear(16,3)
        
        
    def forward(self,x):
        lstm_out,(h_n,h_c) = self.lstm(x,None)
        # only need last output of lstm layer
        lstm_out = self.dropout(lstm_out[:,-1,:])
        h1_out = self.hidden1(lstm_out)
        h1_out = self.dropout(h1_out) 
        out = self.out(h1_out)
        return out


In [11]:
model = RNN()
print(model)

RNN(
  (lstm): LSTM(10, 64, batch_first=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (hidden1): Linear(in_features=64, out_features=16, bias=True)
  (out): Linear(in_features=16, out_features=3, bias=True)
)


In [12]:
# define optimizer and loss function 
optimizer = torch.optim.Adam(model.parameters(), lr=LR,weight_decay=0.0005)
# adject learning rate . when loss don't fall , lr = lr * factor  , min lr = 0.0001
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,factor = 0.9,min_lr=0.0001)
# crossentroy loss 
loss_func = torch.nn.CrossEntropyLoss()

In [13]:

#delcear testing features and labels

test_features = []
for i in range(30,len(test_set)):
    x = test_set[i-30:i][["K value ","D value","William","MACD","SMA","WMA","Momentum","RSI","A/D 0","CCI"]].values
    test_features.append(x.tolist())
test_features = torch.FloatTensor(test_features)
test_labels = []
for i in range(31,len(test_set)+1):
    x = test_set[i-1:i]["Close r/f"]
    test_labels.append(x.tolist())
test_labels = torch.FloatTensor(test_labels).view(-1)


In [ ]:
'''
Training 
'''
LOSS = []
ACC = []
TRAIN_ACC = []
for epoch in range(EPOCH):
    loss_total = 0
    for step,(inputs,targets) in enumerate(train_loader):
        #reshpae inputs to (1,-1)
        inputs = inputs.view(1,-1)
        # minmax scale to range(-1,1)
        inputs = min_max_scaler.fit_transform(inputs)
        #reshape the features to (batch,time_step,input_size)
        inputs = inputs.reshape(-1,TIME_STEP,INPUT_SIZE)
        # transform numpy array to torch tensor
        inputs = torch.Tensor(inputs)
        # start trainnig 
        output = model(inputs)
        # calculate loss  (cross entroy)
        loss = loss_func(output,targets)
        # clear the gradients of all optimized variables(from last training)
        optimizer.zero_grad()
        
        # back propagation
        loss.backward()
        optimizer.step()
        # sum of loss
        loss_total = loss_total + loss
        
        # print training info every 30 steps
        if((step+1) %30 == 0):
            # average of loss in 30 steps
            avg = loss_total / 30
            LOSS.append(avg.tolist())
            
            # calculate the accuracy of training 
            pred_train_y = torch.max(output, 1)[1].data.numpy()
            train_accuracy = float((pred_train_y == targets.numpy()).astype(int).sum()) / float(targets.numpy().size)
            TRAIN_ACC.append(train_accuracy)
            
            # calculate the accuracy of using testing data as inputs
            test_output = model(test_features.view(-1,TIME_STEP,INPUT_SIZE))
            pred_test_y = torch.max(test_output, 1)[1].data.numpy()
            test_accuracy = float((pred_test_y == test_labels.numpy()).astype(int).sum()) / float(test_labels.numpy().size)
            ACC.append(test_accuracy)
            
            # print the epoch , steps , average loss , accuracy 
            print("Epoch: %4d|steps: %4d|Train Avg Loss: %.4f |Train set Accuracy: %.4f |Test set Accuracy: %.4f|lr = %.5f"
                  %(epoch+1,step+1,avg,train_accuracy,test_accuracy,optimizer.param_groups[0]['lr']))
            
            # inital variable
            loss_total = 0
    # updata learning rate
    scheduler.step(loss)
        


Epoch:    1|steps:   30|Train Avg Loss: 1.0137 |Train set Accuracy: 0.1875 |Test set Accuracy: 0.4567|lr = 0.00100
Epoch:    1|steps:   60|Train Avg Loss: 0.9557 |Train set Accuracy: 0.4375 |Test set Accuracy: 0.4471|lr = 0.00100
Epoch:    1|steps:   90|Train Avg Loss: 0.9233 |Train set Accuracy: 0.1875 |Test set Accuracy: 0.4808|lr = 0.00100
Epoch:    1|steps:  120|Train Avg Loss: 0.9095 |Train set Accuracy: 0.4375 |Test set Accuracy: 0.4471|lr = 0.00100
Epoch:    2|steps:   30|Train Avg Loss: 0.8907 |Train set Accuracy: 0.5000 |Test set Accuracy: 0.5529|lr = 0.00100
Epoch:    2|steps:   60|Train Avg Loss: 0.9221 |Train set Accuracy: 0.4375 |Test set Accuracy: 0.4519|lr = 0.00100
Epoch:    2|steps:   90|Train Avg Loss: 0.8877 |Train set Accuracy: 0.5000 |Test set Accuracy: 0.5096|lr = 0.00100
Epoch:    2|steps:  120|Train Avg Loss: 0.8999 |Train set Accuracy: 0.3125 |Test set Accuracy: 0.4327|lr = 0.00100
Epoch:    3|steps:   30|Train Avg Loss: 0.9186 |Train set Accuracy: 0.2500 |Test

Epoch:   19|steps:   30|Train Avg Loss: 0.9085 |Train set Accuracy: 0.5000 |Test set Accuracy: 0.5000|lr = 0.00090
Epoch:   19|steps:   60|Train Avg Loss: 0.9275 |Train set Accuracy: 0.2500 |Test set Accuracy: 0.4712|lr = 0.00090
Epoch:   19|steps:   90|Train Avg Loss: 0.9153 |Train set Accuracy: 0.5625 |Test set Accuracy: 0.5144|lr = 0.00090
Epoch:   19|steps:  120|Train Avg Loss: 0.8915 |Train set Accuracy: 0.6250 |Test set Accuracy: 0.4808|lr = 0.00090
Epoch:   20|steps:   30|Train Avg Loss: 0.8951 |Train set Accuracy: 0.3750 |Test set Accuracy: 0.5481|lr = 0.00090
Epoch:   20|steps:   60|Train Avg Loss: 0.9158 |Train set Accuracy: 0.5000 |Test set Accuracy: 0.5048|lr = 0.00090
Epoch:   20|steps:   90|Train Avg Loss: 0.9230 |Train set Accuracy: 0.5625 |Test set Accuracy: 0.5144|lr = 0.00090
Epoch:   20|steps:  120|Train Avg Loss: 0.8867 |Train set Accuracy: 0.5625 |Test set Accuracy: 0.4808|lr = 0.00090
Epoch:   21|steps:   30|Train Avg Loss: 0.8949 |Train set Accuracy: 0.5625 |Test

Epoch:   37|steps:   30|Train Avg Loss: 0.8921 |Train set Accuracy: 0.6250 |Test set Accuracy: 0.4808|lr = 0.00081
Epoch:   37|steps:   60|Train Avg Loss: 0.8976 |Train set Accuracy: 0.5000 |Test set Accuracy: 0.5048|lr = 0.00081
Epoch:   37|steps:   90|Train Avg Loss: 0.8888 |Train set Accuracy: 0.5000 |Test set Accuracy: 0.4808|lr = 0.00081
Epoch:   37|steps:  120|Train Avg Loss: 0.9317 |Train set Accuracy: 0.4375 |Test set Accuracy: 0.5144|lr = 0.00081
Epoch:   38|steps:   30|Train Avg Loss: 0.9291 |Train set Accuracy: 0.5000 |Test set Accuracy: 0.5144|lr = 0.00081
Epoch:   38|steps:   60|Train Avg Loss: 0.8625 |Train set Accuracy: 0.5625 |Test set Accuracy: 0.5481|lr = 0.00081
Epoch:   38|steps:   90|Train Avg Loss: 0.8697 |Train set Accuracy: 0.3750 |Test set Accuracy: 0.4808|lr = 0.00081
Epoch:   38|steps:  120|Train Avg Loss: 0.9245 |Train set Accuracy: 0.3125 |Test set Accuracy: 0.4712|lr = 0.00081
Epoch:   39|steps:   30|Train Avg Loss: 0.9146 |Train set Accuracy: 0.4375 |Test

Epoch:   55|steps:   30|Train Avg Loss: 0.9018 |Train set Accuracy: 0.4375 |Test set Accuracy: 0.5192|lr = 0.00066
Epoch:   55|steps:   60|Train Avg Loss: 0.8870 |Train set Accuracy: 0.5625 |Test set Accuracy: 0.4808|lr = 0.00066
Epoch:   55|steps:   90|Train Avg Loss: 0.9194 |Train set Accuracy: 0.5625 |Test set Accuracy: 0.4808|lr = 0.00066
Epoch:   55|steps:  120|Train Avg Loss: 0.8936 |Train set Accuracy: 0.6875 |Test set Accuracy: 0.5096|lr = 0.00066
Epoch:   56|steps:   30|Train Avg Loss: 0.8998 |Train set Accuracy: 0.6250 |Test set Accuracy: 0.4567|lr = 0.00066
Epoch:   56|steps:   60|Train Avg Loss: 0.8825 |Train set Accuracy: 0.3125 |Test set Accuracy: 0.5481|lr = 0.00066
Epoch:   56|steps:   90|Train Avg Loss: 0.9187 |Train set Accuracy: 0.4375 |Test set Accuracy: 0.4952|lr = 0.00066
Epoch:   56|steps:  120|Train Avg Loss: 0.9066 |Train set Accuracy: 0.3750 |Test set Accuracy: 0.4471|lr = 0.00066
Epoch:   57|steps:   30|Train Avg Loss: 0.8934 |Train set Accuracy: 0.4375 |Test

Epoch:   73|steps:   30|Train Avg Loss: 0.8999 |Train set Accuracy: 0.3750 |Test set Accuracy: 0.5337|lr = 0.00053
Epoch:   73|steps:   60|Train Avg Loss: 0.8763 |Train set Accuracy: 0.6250 |Test set Accuracy: 0.4712|lr = 0.00053
Epoch:   73|steps:   90|Train Avg Loss: 0.9417 |Train set Accuracy: 0.3750 |Test set Accuracy: 0.5096|lr = 0.00053
Epoch:   73|steps:  120|Train Avg Loss: 0.9142 |Train set Accuracy: 0.5625 |Test set Accuracy: 0.4952|lr = 0.00053
Epoch:   74|steps:   30|Train Avg Loss: 0.9571 |Train set Accuracy: 0.3750 |Test set Accuracy: 0.4904|lr = 0.00053
Epoch:   74|steps:   60|Train Avg Loss: 0.8812 |Train set Accuracy: 0.4375 |Test set Accuracy: 0.5144|lr = 0.00053
Epoch:   74|steps:   90|Train Avg Loss: 0.8893 |Train set Accuracy: 0.4375 |Test set Accuracy: 0.4952|lr = 0.00053
Epoch:   74|steps:  120|Train Avg Loss: 0.9002 |Train set Accuracy: 0.3750 |Test set Accuracy: 0.5096|lr = 0.00053
Epoch:   75|steps:   30|Train Avg Loss: 0.8933 |Train set Accuracy: 0.6250 |Test

...

Epoch: 3529|steps:   30|Train Avg Loss: 0.9001 |Train set Accuracy: 0.5000 |Test set Accuracy: 0.5000|lr = 0.00010
Epoch: 3529|steps:   60|Train Avg Loss: 0.9232 |Train set Accuracy: 0.4375 |Test set Accuracy: 0.4904|lr = 0.00010
Epoch: 3529|steps:   90|Train Avg Loss: 0.9186 |Train set Accuracy: 0.4375 |Test set Accuracy: 0.4904|lr = 0.00010
Epoch: 3529|steps:  120|Train Avg Loss: 0.8690 |Train set Accuracy: 0.5625 |Test set Accuracy: 0.4808|lr = 0.00010
Epoch: 3530|steps:   30|Train Avg Loss: 0.9147 |Train set Accuracy: 0.5625 |Test set Accuracy: 0.5192|lr = 0.00010
Epoch: 3530|steps:   60|Train Avg Loss: 0.8728 |Train set Accuracy: 0.6875 |Test set Accuracy: 0.5192|lr = 0.00010
Epoch: 3530|steps:   90|Train Avg Loss: 0.8845 |Train set Accuracy: 0.5000 |Test set Accuracy: 0.5096|lr = 0.00010
Epoch: 3530|steps:  120|Train Avg Loss: 0.9109 |Train set Accuracy: 0.5625 |Test set Accuracy: 0.4760|lr = 0.00010
Epoch: 3531|steps:   30|Train Avg Loss: 0.8882 |Train set Accuracy: 0.4375 |Test

Epoch: 3547|steps:   30|Train Avg Loss: 0.8796 |Train set Accuracy: 0.5625 |Test set Accuracy: 0.5288|lr = 0.00010
Epoch: 3547|steps:   60|Train Avg Loss: 0.8699 |Train set Accuracy: 0.5625 |Test set Accuracy: 0.4519|lr = 0.00010
Epoch: 3547|steps:   90|Train Avg Loss: 0.9318 |Train set Accuracy: 0.2500 |Test set Accuracy: 0.5000|lr = 0.00010
Epoch: 3547|steps:  120|Train Avg Loss: 0.9202 |Train set Accuracy: 0.5000 |Test set Accuracy: 0.5192|lr = 0.00010
Epoch: 3548|steps:   30|Train Avg Loss: 0.9074 |Train set Accuracy: 0.8125 |Test set Accuracy: 0.4904|lr = 0.00010
Epoch: 3548|steps:   60|Train Avg Loss: 0.9143 |Train set Accuracy: 0.5625 |Test set Accuracy: 0.4712|lr = 0.00010
Epoch: 3548|steps:   90|Train Avg Loss: 0.8606 |Train set Accuracy: 0.3125 |Test set Accuracy: 0.4952|lr = 0.00010
Epoch: 3548|steps:  120|Train Avg Loss: 0.9119 |Train set Accuracy: 0.6250 |Test set Accuracy: 0.4375|lr = 0.00010
Epoch: 3549|steps:   30|Train Avg Loss: 0.9069 |Train set Accuracy: 0.5000 |Test

Epoch: 3565|steps:   30|Train Avg Loss: 0.8763 |Train set Accuracy: 0.4375 |Test set Accuracy: 0.5288|lr = 0.00010
Epoch: 3565|steps:   60|Train Avg Loss: 0.9309 |Train set Accuracy: 0.6250 |Test set Accuracy: 0.5192|lr = 0.00010
Epoch: 3565|steps:   90|Train Avg Loss: 0.9230 |Train set Accuracy: 0.5625 |Test set Accuracy: 0.5048|lr = 0.00010
Epoch: 3565|steps:  120|Train Avg Loss: 0.8809 |Train set Accuracy: 0.3750 |Test set Accuracy: 0.4904|lr = 0.00010
Epoch: 3566|steps:   30|Train Avg Loss: 0.8999 |Train set Accuracy: 0.3750 |Test set Accuracy: 0.4904|lr = 0.00010
Epoch: 3566|steps:   60|Train Avg Loss: 0.8956 |Train set Accuracy: 0.5625 |Test set Accuracy: 0.5096|lr = 0.00010
Epoch: 3566|steps:   90|Train Avg Loss: 0.8996 |Train set Accuracy: 0.4375 |Test set Accuracy: 0.4952|lr = 0.00010
Epoch: 3566|steps:  120|Train Avg Loss: 0.9045 |Train set Accuracy: 0.7500 |Test set Accuracy: 0.4712|lr = 0.00010
Epoch: 3567|steps:   30|Train Avg Loss: 0.9347 |Train set Accuracy: 0.5625 |Test

Epoch: 3583|steps:   30|Train Avg Loss: 0.9147 |Train set Accuracy: 0.2500 |Test set Accuracy: 0.5144|lr = 0.00010
Epoch: 3583|steps:   60|Train Avg Loss: 0.9000 |Train set Accuracy: 0.6250 |Test set Accuracy: 0.4760|lr = 0.00010
Epoch: 3583|steps:   90|Train Avg Loss: 0.9000 |Train set Accuracy: 0.5625 |Test set Accuracy: 0.4904|lr = 0.00010
Epoch: 3583|steps:  120|Train Avg Loss: 0.8844 |Train set Accuracy: 0.6250 |Test set Accuracy: 0.4856|lr = 0.00010
Epoch: 3584|steps:   30|Train Avg Loss: 0.9039 |Train set Accuracy: 0.5625 |Test set Accuracy: 0.4808|lr = 0.00010
Epoch: 3584|steps:   60|Train Avg Loss: 0.9081 |Train set Accuracy: 0.5625 |Test set Accuracy: 0.4760|lr = 0.00010
Epoch: 3584|steps:   90|Train Avg Loss: 0.8776 |Train set Accuracy: 0.3125 |Test set Accuracy: 0.4904|lr = 0.00010
Epoch: 3584|steps:  120|Train Avg Loss: 0.9313 |Train set Accuracy: 0.5000 |Test set Accuracy: 0.5000|lr = 0.00010
Epoch: 3585|steps:   30|Train Avg Loss: 0.9079 |Train set Accuracy: 0.3125 |Test

Epoch: 3601|steps:   30|Train Avg Loss: 0.9078 |Train set Accuracy: 0.5625 |Test set Accuracy: 0.5096|lr = 0.00010
Epoch: 3601|steps:   60|Train Avg Loss: 0.9307 |Train set Accuracy: 0.2500 |Test set Accuracy: 0.4856|lr = 0.00010
Epoch: 3601|steps:   90|Train Avg Loss: 0.8965 |Train set Accuracy: 0.3750 |Test set Accuracy: 0.4615|lr = 0.00010
Epoch: 3601|steps:  120|Train Avg Loss: 0.8696 |Train set Accuracy: 0.3750 |Test set Accuracy: 0.4952|lr = 0.00010
Epoch: 3602|steps:   30|Train Avg Loss: 0.9119 |Train set Accuracy: 0.3125 |Test set Accuracy: 0.5000|lr = 0.00010
Epoch: 3602|steps:   60|Train Avg Loss: 0.9049 |Train set Accuracy: 0.5000 |Test set Accuracy: 0.4663|lr = 0.00010
Epoch: 3602|steps:   90|Train Avg Loss: 0.9114 |Train set Accuracy: 0.5000 |Test set Accuracy: 0.5144|lr = 0.00010
Epoch: 3602|steps:  120|Train Avg Loss: 0.8688 |Train set Accuracy: 0.5000 |Test set Accuracy: 0.5144|lr = 0.00010
Epoch: 3603|steps:   30|Train Avg Loss: 0.9473 |Train set Accuracy: 0.3125 |Test

In [ ]:
#64 32 16 3
#dynamic adject learning rate 0.001 ~0.0001
#(dropout,LL2)

# 0.2, 0.0005 = underfitting ?  working?
# 0.2,0.0006 => overfitting  epoch 6000